In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd

In [10]:
# Geckodriver yolu
geckodriver_path = "./drivers/geckodriver"

# Firefox service nesnesini oluştur
service = FirefoxService(executable_path=geckodriver_path)


# Firefox tarayıcısını service ile başlat
driver = webdriver.Firefox(service=service)
# Hedef web sayfasına git

macKolikUrl="https://www.mackolik.com/futbol/canli-sonuclar"
driver.get(macKolikUrl)

#time.sleep(5)
wait = WebDriverWait(driver, 10)
cookies=wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]')))
cookies.click()

#click for ordering by matches start time
#clock=wait.until(EC.element_to_be_clickable((By.XPATH, "//span[.='Saat']")))
#clock.click()

dateIcon = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.widget-dateslider__datepicker-toggle')))

# Click on the element
dateIcon.click()
butunTakvim = wait.until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]')))
html_contentTakvim = butunTakvim.get_attribute('outerHTML')
# print(html_contentTakvim)



# BeautifulSoup ile HTML içeriğini parse et
soup = BeautifulSoup(html_contentTakvim, 'html.parser')

# Günü ve Yılı al
day = int(soup.find('td', {'class': 'widget-datepicker__calendar-body-cell--selected-day'}).text.strip())
year = int(soup.find('div', {'class': 'widget-datepicker__selector--year'}).find('div', {'class': 'widget-datepicker__value'}).text.strip())

#print(f"day: {day}, year: {year}")

current_year = int(driver.find_element(By.XPATH, '/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[1]/div[2]/div[2]').text)
previous_year_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='widget-dateslider__datepicker widget-dateslider__datepicker--open']/div/div/div[2]/div[1]")))

# Hedef yılı belirle (örneğin, 2020)
target_year = 2022

# Hedef yıla kadar önceki yıl düğmesine tıkla
while current_year > target_year:
    #print(str(target_year)+ "target year")
    previous_year_button.click()
    #time.sleep(1)
    current_year -= 1
    #print(current_year)
    # Yeni yılı kontrol et
    current_year = int(driver.find_element(By.XPATH, '/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[1]/div[2]/div[2]').text)
    #print(current_year)

    
# calc of months
previous_month_button= wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[1]/div[1]/div[1]")))
#wait = WebDriverWait(driver, 10)  # Adjust the timeout as needed
element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[1]/div[1]/div[2]")))
month_str = element.text.strip()[:3]

#print(month_str)
# Map the month abbreviation to a numeric value
month_mapping = {'OCA': 1, 'ŞUB': 2, 'MAR': 3, 'NIS': 4, 'MAY': 5, 'HAZ': 6, 'TEM': 7, 'AĞU': 8, 'EYL': 9, 'EKI': 10, 'KAS': 11, 'ARA': 12}
#print(month_str+"month_str")
month_numeric = month_mapping.get(month_str)

target_month=12
month_numeric = month_mapping.get(month_str)


# Loop until you reach the target month
while month_numeric != target_month:
    #print(str(month_numeric)+"month_numeric")
    #print(str(target_month)+"target_month")

    
    previous_month_button.click()
    #time.sleep(1)
    
    element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[5]/div/main/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div/div[1]/div[1]/div[2]")))
    month_str = element.text.strip()[:3]
    #print(month_str+"month_str")
    month_numeric = month_mapping.get(month_str)
    #print(month_numeric)
    #print(target_month)
    
target_day=14
day_element = driver.find_element(By.XPATH, f"//td[@class='widget-datepicker__calendar-body-cell' and text()='{target_day}']")
#time.sleep(19)
day_element.click()
#time.sleep()


# wait for page loaded
WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.XPATH, "//body"))
)

#daily all  matches rows container elements xpath
container_xpath = "//div[@class='match-row__match-content']"  #(//div[@class='match-row__match-content'])[1]


wait_time = 15

#daily all  matches rows container ELEMENT
allLinksElements = WebDriverWait(driver, wait_time).until(
    EC.visibility_of_all_elements_located((By.XPATH, container_xpath))
)

# Container elementini bul (alternative)
#allLinksElements = driver.find_elements(By.XPATH, container_xpath)

#dataframe all bettings odds one match
#df_all = pd.DataFrame()
count = 0

while True:
    try:
        # Container elementini bul
        #allLinksElements = driver.find_elements(By.XPATH, container_xpath)

        # If all links are processed, break out of the loop
        #if not allLinksElements:
            #break

        # Iterate through each element and click
        for i, element in enumerate(allLinksElements, start=1):
            try:
                # precautionary for page loading
                dynamic_xpath = f"(//div[@class='match-row__match-content'])[{i}]"
                #elementx = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, dynamic_xpath)))
                elementx = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, dynamic_xpath)))

                print(f"(try Worked... {str(i)})")
                element.click()

                # Switch to the newly opened tab (assuming it's the last tab)
                driver.switch_to.window(driver.window_handles[-1])

                print(f"(went new page... {str(i)})")
                #time.sleep(1)
                
                # Perform actions in the new tab
                iddiaButton = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div[1]/div[2]/div/a[3]")))

                # get class atribute values...
                tx = iddiaButton.get_attribute('class')
                
                # Check if the iddiaButton is disabled
                if "disabled" not in tx:
                    print('Button is not disabled. Proceeding...')
                    iddiaButton.click()
                    time.sleep(1)
                    
                    #top scorborad table element
                    macScorboard = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='p0c-soccer-match-details-header']")))
                    
                    #main betting table element
                    macDetailsData = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div[2]/main/div[1]/div[2]/div/div[2]/div")))
                    
                    #scoreboard as a string
                    scoreboard_text = macScorboard.text
                    lines = scoreboard_text.strip().split('\n')
                    print(lines)
                    
                    # Generate column names based on the number of columns in each row
                    #max_columns = max(len(line.split()) for line in lines)
                    #column_names = [f"Column_{i}" for i in range(1, max_columns + 1)]

                    # generate DataFrame 
                    #df = pd.DataFrame([line.split() for line in lines], columns=column_names)
                    #print(df)
                    
                    #get match details as string text
                    mac_Details_Data=macDetailsData.text
                    detail_lines=mac_Details_Data.strip().split('\n')
                    #print match details...
                    #print(detail_lines)                                            
                    
                    # Close the current tab
                    driver.close()
                    print(f"(gone new page closed.. {str(i)})")
                    #time.sleep(1)
                    # Switch back to the original tab
                    driver.switch_to.window(driver.window_handles[0])

                    time.sleep(1)
                else:
                    print('Button is disabled. Skipping...')
                    # Switch back to the original tab
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

            except StaleElementReferenceException:
                count += 1
                print('<><><><><><', count)
                print("Stale Element Reference Exception. Retrying.")
                continue

    except StaleElementReferenceException:
        print("Stale Element Reference Exception during initial find. Retrying after refreshing the page.")
        driver.refresh()
        continue

(try Worked... 1)
(went new page... 1)
Button is not disabled. Proceeding...
['Fransa', 'Fas', '2', 'MS', '0', '(İY 1 - 0)', '14-12-22 | Dünya Kupası | Stat: Al Bayt Stadium (68294)']
(gone new page closed.. 1)
(try Worked... 2)
(went new page... 2)
Button is not disabled. Proceeding...
['Portimonense', 'Gil Vicente', '0', 'MS', '3', '(İY 0 - 1)', '14-12-22 | Lig Kupası | 3. Hafta | Stat: Estádio Municipal de Portimão (213)']
(gone new page closed.. 2)
(try Worked... 3)
(went new page... 3)
Button is not disabled. Proceeding...
['Covilha', 'Nacional', '1', 'MS', '2', '(İY 1 - 0)', '14-12-22 | Lig Kupası | 3. Hafta | Stat: Estádio Municipal José Santos Pinto (94)']
(gone new page closed.. 3)
(try Worked... 4)
(went new page... 4)
Button is not disabled. Proceeding...
['Vietnam', 'Filipinler', '1', 'MS', '0', '14-12-22 | Hazırlık Maçları | Stat: Sân vận động Hàng Đẫy (Hang Day Stadium)']
(gone new page closed.. 4)
(try Worked... 5)
(went new page... 5)


TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16
